In [133]:
#Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import time
import pandas as pd
import requests
from pprint import pprint
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [134]:
#Population pull
population_url = race_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,P001001&for=state:*&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"
population = requests.get(population_url).json()

In [145]:
population_df = pd.DataFrame(population)
population_df.columns = population_df.iloc[0]
population_df = population_df.reindex(population_df.index.drop(0))
population_df = population_df.rename(columns={"NAME":"State","P001001":"population","state":"count"})
population_df = population_df[['State','population']]
population_df.head()

,State,population
1,Alabama,4779736
2,Alaska,710231
3,Arizona,6392017
4,Arkansas,2915918
5,California,37253956


In [136]:
#Race pull
race_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,PCT012C001,PCT012D001,PCT012B001,PCT012H001,PCT012E001,PCT012F001,PCT012G001,PCT012A001&for=state:*&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"
race = requests.get(race_url).json()

In [146]:
race_df = pd.DataFrame(race)
race_df.columns = race_df.iloc[0]
race_df = race_df.reindex(race_df.index.drop(0))
race_df = race_df.rename(columns={"NAME":"State","PCT012C001":"american_indian_and_alaska_native","PCT012D001":"asian","PCT012B001":"black_or_african_american","PCT012H001":"hispanic_or_latino","PCT012E001":"native_hawaiian_other_pacific_islander","PCT012F001":"some_other_race","PCT012G001":"two_or_more_races","PCT012A001":"white"})
del race_df['state']
race_df.head()

,State,american_indian_and_alaska_native,asian,black_or_african_american,hispanic_or_latino,native_hawaiian_other_pacific_islander,some_other_race,two_or_more_races,white
1,Alabama,28218,53595,1251311,185602,3057,96910,71251,3275394
2,Alaska,104871,38135,23263,39249,7409,11102,51875,473576
3,Arizona,296529,176695,259008,1895149,12648,761716,218300,4667121
4,Arkansas,22248,36102,449895,186050,5863,99571,57010,2245229
5,California,362801,4861007,2299072,14013719,144386,6317372,1815384,21453934


In [138]:
#Household Size pull
house_size_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,H013002,H013003,H013004,H013005,H013006,H013007,H013008&for=state:*&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"
size = requests.get(house_size_url).json()

In [147]:
household_size_df = pd.DataFrame(size)
household_size_df.columns = household_size_df.iloc[0]
household_size_df = household_size_df.reindex(household_size_df.index.drop(0))
household_size_df = household_size_df.rename(columns={"NAME":"State","H013002":"1_persons_or_more","H013003":"2_persons_or_more","H013004":"3_persons_or_more","H013005":"4_persons_or_more","H013006":"5_persons_or_more","H013007":"6_persons_or_more","H013008":"7_persons_or_more"})
del household_size_df['state']
household_size_df.head()

,State,1_persons_or_more,2_persons_or_more,3_persons_or_more,4_persons_or_more,5_persons_or_more,6_persons_or_more,7_persons_or_more
1,Alabama,516696,632291,322941,245326,106771,37510,22256
2,Alaska,66073,83852,41891,34124,18122,7546,6450
3,Arizona,621008,814955,344326,296655,164944,77095,62007
4,Arkansas,310792,402111,186450,142819,65311,24496,15105
5,California,2929442,3653802,2043812,1883451,1040408,507471,519112


#API to pull income
https://api.census.gov/data/2010/acs/acs5/variables.html

#Variable used
B06011_002E	Estimate!!Median income in the past 12 months!!Total!!Born in state of residence	MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2010 INFLATION-ADJUSTED DOLLARS) BY PLACE OF BIRTH IN THE UNITED STATES

api.census.gov/data/2010/acs/acs5?key=YOUR KEY GOES HERE&get=B02001_001E,NAME&for=state:06,36

In [140]:
#Median Income last 12 months pull
income_url = "https://api.census.gov/data/2010/acs/acs5?key=59a41ecf0dff091bd991b35c87176ffd56b8107d&get=B06011_002E,NAME&for=state:*"
income = requests.get(income_url).json()

In [141]:
income_df = pd.DataFrame(income)
income_df.columns = income_df.iloc[0]
income_df = income_df.reindex(income_df.index.drop(0))
income_df = income_df.rename(columns={"NAME":"State","B06011_002E":"med_income_ltm"})
income_df = income_df[['State','med_income_ltm']]
income_df.tail()

,State,med_income_ltm
48,Washington,27280
49,West Virginia,20071
50,Wisconsin,26954
51,Wyoming,25498
52,Puerto Rico,None


In [142]:
#Age pull
age_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,P013003,P013002&for=state:*&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"
age = requests.get(age_url).json()

In [148]:
age_df = pd.DataFrame(age)
age_df.columns = age_df.iloc[0]
age_df = age_df.reindex(age_df.index.drop(0))
age_df = age_df.rename(columns={"NAME":"State","P013003":"med_age_female","P013002":"med_age_male"})
del age_df['state']
age_df.head()

,State,med_age_female,med_age_male
1,Alabama,39.1,36.4
2,Alaska,34.1,33.6
3,Arizona,37.2,34.8
4,Arkansas,38.7,36.1
5,California,36.3,34.0


In [150]:
#Occupancy pull
occupancy_url = "https://api.census.gov/data/2010/dec/sf1?get=NAME,H004003,H004002,H004004&for=state:*&key=59a41ecf0dff091bd991b35c87176ffd56b8107d"
occupancy = requests.get(occupancy_url).json()

In [152]:
occupancy_df = pd.DataFrame(occupancy)
occupancy_df.columns = occupancy_df.iloc[0]
occupancy_df = occupancy_df.reindex(occupancy_df.index.drop(0))
occupancy_df = occupancy_df.rename(columns={"NAME":"State","H004003":"owned_free_clear","H004002":"owned_mortgage","H004004":"rent"})
del occupancy_df['state']
occupancy_df.head()

,State,owned_free_clear,owned_mortgage,rent
1,Alabama,480579,832010,571202
2,Alaska,50753,112012,95293
3,Arizona,439025,1132662,809303
4,Arkansas,298502,469654,378928
5,California,1570026,5465345,5542127


In [162]:
#Merge data-frames
merged_df = population_df.merge(race_df.merge(household_size_df, on='State'), on='State') 
merged_df_2 = income_df.merge(age_df.merge(occupancy_df, on='State'), on='State') 
merged_final = merged_df.merge(merged_df_2, on="State")

In [163]:
merged_final.tail()

,State,population,american_indian_and_alaska_native,asian,black_or_african_american,hispanic_or_latino,native_hawaiian_other_pacific_islander,some_other_race,two_or_more_races,white,...,4_persons_or_more,5_persons_or_more,6_persons_or_more,7_persons_or_more,med_income_ltm,med_age_female,med_age_male,owned_free_clear,owned_mortgage,rent
47,Washington,6724540,103869,481067,240042,755790,40475,349799,312926,5196362,...,338260,151893,62772,44903,27280,38.3,36.2,428656,1245264,946156
48,West Virginia,1852994,3787,12406,63124,22268,428,6119,27142,1739988,...,89437,35982,11812,6076,20071,42.6,40.1,267491,293522,202818
49,Wisconsin,5686986,54526,129234,359148,336056,1827,135867,104317,4902067,...,284532,124387,44504,27052,26954,39.6,37.3,469459,1082099,728210
50,Wyoming,563626,13336,4426,4748,50231,427,17049,12361,511279,...,26654,12444,4825,2865,25498,37.7,36.0,57296,99781,69802
51,Puerto Rico,3725789,19839,6831,461498,3688455,370,289905,122246,2825100,...,227988,99560,31434,17719,None,38.6,35.1,554508,431657,390366


In [164]:
merged_final.to_excel("census_data.xlsx")